In [1]:
import tensorflow as tf
import dataset_helpers as dh
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from importlib import reload
reload(dh)

<module 'dataset_helpers' from '/home/DeepMRI/src/dataset_helpers.py'>

In [2]:

train_dataset = dh.load_dataset('../datasets/brats2015-Train-all_training_crop_mri',
                                mri_type=['MR_T1c', 'MR_T2', 'MR_Flair'],
                                random_crop=[160,160,3],
                                batch_size=64,
                                prefetch_buffer=1,
                                shuffle_buffer=128,
                                cache=True,
                                cast_to=np.float32,
                                clip_labels_to=1.0,
                                infinite = False)

In [ ]:
def load_all_records(dataset, limit_batches=None):
    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    batches = []
    with tf.Session() as sess: 
        while True:
            try:
                if len(batches) == (limit_batches or -1):
                    break
                batches += sess.run([next_batch])
            except Exception as e:
                print(e)
                break
    return batches
records = load_all_records(train_dataset)

End of sequence
	 [[node IteratorGetNext_1 (defined at <ipython-input-4-a7b5ba1e0f75>:3)  = IteratorGetNext[output_shapes=[[?], [?], [?], [?], [?], ..., [?], [?], [?], [?,160,160,3], [?,160,160,1]], output_types=[DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_STRING, ..., DT_INT64, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_1)]]

Caused by op 'IteratorGetNext_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  F

In [31]:
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact, fixed
def view_slice(records, r, s):
    batch = records[r]
    mri = batch['mri'][s]
    gt = batch['seg'][s]
    
    plt.figure(figsize=(16,9))
    for i in range(mri.shape[-1]):
        plt.subplot(1, mri.shape[-1]+1, i+1)
        plt.imshow(mri[:,:,i])
    plt.subplot(1, mri.shape[-1]+1, mri.shape[-1]+1)
    plt.imshow(gt[:,:,0])
interact(view_slice, records=fixed(records), r=IntSlider(min=0, max=len(records)-1), s=IntSlider(min=0, max=63))

interactive(children=(IntSlider(value=0, description='r', max=2), IntSlider(value=0, description='s', max=63),…

<function __main__.view_slice(records, r, s)>

In [32]:
records[0]['seg'][0].shape

(160, 160, 1)

In [4]:
def count_all_records(dataset):
    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    i = 0
    with tf.Session() as sess: 
        while True:
            try:
                sess.run([next_batch])
                i += 1
                print(i)
            except Exception as e:
                print(e)
                break
    return i
records = count_all_records(train_dataset)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
